# Recurrent Network

This notebook shows how to create a recurrent network

### Import all the needed modules

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_absolute_error

### Generate random values and calculate the label using the toy problem

The model shold be able to learn how to calculate the standard deviation of a sequence

To define data with different lengths, we set the unused parts of the data as -1.

In [ ]:
N_SAMPLES = 100000
N_TEST = 1000
MAX_TIMESTEPS = 10
MASK_VALUE = -1

train_X = np.random.uniform(size=(N_SAMPLES, MAX_TIMESTEPS, 1))
train_L = np.random.randint(2, MAX_TIMESTEPS, N_SAMPLES)

test_X = np.random.uniform(size=(N_TEST, MAX_TIMESTEPS, 1))
test_L = np.random.randint(2, MAX_TIMESTEPS, N_TEST)

In [ ]:
for i in range(N_SAMPLES):
    train_X[i, train_L[i]:] = MASK_VALUE

In [ ]:
for i in range(N_TEST):
    test_X[i, test_L[i]:] = MASK_VALUE

In [ ]:
train_y = np.ma.masked_array(train_X, train_X==MASK_VALUE).std(axis=1).data
test_y = np.ma.masked_array(test_X, test_X==MASK_VALUE).std(axis=1).data

### Define the connections of the nodes in the network and verify the model

The ```Masking``` layer removes the numbers whose values are equal to -1. This allows us to tell the network that the data lengths are different.

The ```return_sequence``` flag defines if the LSTM should return only the last or the full sequence

In [ ]:
input_ = tf.keras.Input(shape=(None, 1))
masked = tf.keras.layers.Masking(MASK_VALUE)(input_)
lstm1 = tf.keras.layers.LSTM(32, return_sequences=True)(masked)
lstm2 = tf.keras.layers.LSTM(32)(lstm1)
output = tf.keras.layers.Dense(1)(lstm2)

model = tf.keras.Model(inputs=input_, outputs=output)
model.summary()

### Compile the model by providing the optimization algorithm and the loss function

In [ ]:
model.compile('adam', 'mse')

### Train the model for 3 epochs

In [ ]:
hist = model.fit(train_X, train_y, epochs=3)

### View the predictions on the test set

In [ ]:
prediction = model.predict(test_X)

In [ ]:
prediction[:5]

In [ ]:
test_y[:5]

In [ ]:
mean_absolute_error(test_y, prediction)